In [1]:
import numpy as np
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine, text
import pandas as pd
import json
from naivebayes import NaiveBayesTextClassifier
from sklearn.metrics import classification_report

In [2]:
load_dotenv()

True

In [3]:
username = os.getenv('SNOWFLAKE_USER')
password = os.getenv('SNOWFLAKE_PASSWORD')
account = os.getenv('SNOWFLAKE_ACCOUNT')
warehouse = os.getenv('SNOWFLAKE_WAREHOUSE')
database = os.getenv('SNOWFLAKE_DATABASE')
schema = os.getenv('SNOWFLAKE_SCHEMA')

In [4]:
engine = create_engine(
    f'snowflake://{username}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'
)

/Users/flo/Desktop/ITU/MSc/3rd_semester/ADS/.venv/lib/python3.12/site-packages/snowflake/sqlalchemy/base.py:1068: SAWarning: The GenericFunction 'flatten' is already registered and is going to be overridden.
  functions.register_function("flatten", flatten)


In [5]:
query = "SELECT * FROM Training"

In [6]:
with engine.connect() as connection:
    result = connection.execute(text(query))
    rows = result.fetchall()

In [7]:
train = pd.DataFrame([json.loads(rows[i][0]) for i in range(len(rows))])

In [8]:
train = train[train['label'].isin([0,1])]

In [10]:
query = "SELECT * FROM Test LIMIT 500"

In [11]:
with engine.connect() as connection:
    result = connection.execute(text(query))
    rows = result.fetchall()

In [12]:
test = pd.DataFrame([json.loads(rows[i][0]) for i in range(len(rows))])

In [14]:
test = test[test['label'].isin([0,1])]

In [9]:
classifier = NaiveBayesTextClassifier()
classifier.fit(train['text'], train['label'])

In [21]:
type(classifier.word_probabilities)

dict

In [22]:
def convert_ndarrays(obj):
    if isinstance(obj, dict):
        return {key: convert_ndarrays(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_ndarrays(item) for item in obj]
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

In [23]:
converted_data = convert_ndarrays(classifier.word_probabilities)


In [24]:
with open('wordprobabilities.json', 'w') as json_file:
    json.dump(converted_data, json_file, indent=4)

In [15]:
y_pred = classifier.predict(test['text'])

In [16]:
y_true = list(test['label'])

In [17]:
print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.64      0.54      0.58        87
           1       0.70      0.77      0.73       119

    accuracy                           0.67       206
   macro avg       0.67      0.66      0.66       206
weighted avg       0.67      0.67      0.67       206



##### Naive Bayes Python implementation

In [ ]:
# priors = {}
# for i,j in Counter(train['label']).items():
#     priors[i] = j/len(train['label'])

In [ ]:
# vocab_per_label_dict = {}
# for label, vector in zip(list(train['label']), X.toarray()):
#     if label in vocab_per_label_dict:
#         vocab_per_label_dict[label] += vector
#     else:
#         vocab_per_label_dict[label] = vector


In [ ]:
# total_per_label = {}
# for i, j in vocab_per_label_dict.items():
#     total_per_label[i] = np.sum(j)  

In [ ]:
# word_probabilities = {}
# for i,j in vocab_per_label_dict.items():
#     word_prob_list = []
#     for k in j:
#         if k != 0:
#             prob = k/total_per_label[i]
#             word_prob_list.append(prob)
#         else:
#             word_prob_list.append(0)
#     word_probabilities[i] = np.array(word_prob_list)

In [ ]:
# results = dict()
# for label in word_probabilities.keys():
#     test = []
#     for i,j in zip(word_probabilities[label],y.toarray()[0]):
#         if i == 0:
#             test.append(1)
#         else:
#             test.append(i**j)
#     results[label] = priors[label]*math.prod(test)

In [ ]:
# max_key = max(results, key=lambda k: results[k])

In [ ]:
# class NaiveBayesTextClassifier:
#     def __init__(self):
#         self.vectorizer = CountVectorizer()
#         self.priors = {}
#         self.vocab_per_label_dict = {}
#         self.total_per_label = {}
#         self.word_probabilities = {}
    
#     def fit(self, train_texts, train_labels):
#         X = self.vectorizer.fit_transform(train_texts)
#         self.feature_names = self.vectorizer.get_feature_names_out()
        
#         label_counts = Counter(train_labels)
#         self.priors = {label: count / len(train_labels) for label, count in label_counts.items()}
        
#         for label, vector in zip(train_labels, X.toarray()):
#             if label in self.vocab_per_label_dict:
#                 self.vocab_per_label_dict[label] += vector
#             else:
#                 self.vocab_per_label_dict[label] = vector
        
#         self.total_per_label = {label: np.sum(vector) for label, vector in self.vocab_per_label_dict.items()}
        
#         self.word_probabilities = {}
#         for label, vector in self.vocab_per_label_dict.items():
#             word_prob_list = [count / self.total_per_label[label] if count != 0 else 0 for count in vector]
#             self.word_probabilities[label] = np.array(word_prob_list)
    
#     def predict(self, test_texts):
#             # Vectorize the test data
#             X_test = self.vectorizer.transform(test_texts)
#             predictions = []
            
#             # Iterate over each test document
#             for test_vector in X_test.toarray():
#                 results = {}
                
#                 # For each label, compute the likelihood for the current test document
#                 for label in self.word_probabilities.keys():
#                     class_probabilities = []
#                     for i, word_count in enumerate(test_vector):
#                         word_prob = self.word_probabilities[label][i]
#                         if word_prob == 0:
#                             class_probabilities.append(1)
#                         else:
#                             class_probabilities.append(word_prob ** word_count)
                    
#                     # Multiply all word probabilities and store the result for the label
#                     results[label] = math.prod(class_probabilities)
                
#                 # Select the label with the highest likelihood
#                 max_key = max(results, key=lambda k: results[k])
#                 predictions.append(max_key)
            
#             return predictions

In [ ]:
# classifier = NaiveBayesTextClassifier()
# classifier.fit(train['text'], train['label'])

In [ ]:
# prediction = classifier.predict(test['text'])

In [ ]:
# prediction

In [ ]:
# import math
# import numpy as np
# from collections import Counter
# from sklearn.feature_extraction.text import CountVectorizer
# from functools import reduce
# import operator

# class NaiveBayesTextClassifier:
#     def __init__(self):
#         self.vectorizer = CountVectorizer()
#         self.priors = {}
#         self.vocab_per_label_dict = {}
#         self.total_per_label = {}
#         self.word_probabilities = {}
    
#     def fit(self, train_texts, train_labels):
#         X = self.vectorizer.fit_transform(train_texts)
#         self.feature_names = self.vectorizer.get_feature_names_out()
        
#         label_counts = Counter(train_labels)
#         self.priors = {label: count / len(train_labels) for label, count in label_counts.items()}
        
#         for label, vector in zip(train_labels, X.toarray()):
#             if label in self.vocab_per_label_dict:
#                 self.vocab_per_label_dict[label] += vector
#             else:
#                 self.vocab_per_label_dict[label] = vector
        
#         self.total_per_label = {label: np.sum(vector) for label, vector in self.vocab_per_label_dict.items()}
        
#         self.word_probabilities = {}
#         for label, vector in self.vocab_per_label_dict.items():
#             word_prob_list = [(count + 1) / (self.total_per_label[label] + len(self.feature_names)) 
#                               for count in vector]  # Laplace Smoothing

#             # word_prob_list = [count / self.total_per_label[label] if count != 0 else 0 for count in vector]

#             self.word_probabilities[label] = np.array(word_prob_list)
    
#     def predict(self, test_texts):
#         X_test = self.vectorizer.transform(test_texts)
#         predictions = []
        
#         for test_vector in X_test.toarray():
#             results = {}
            
#             for label in self.word_probabilities.keys():
#                 class_probabilities = []
#                 for i, word_count in enumerate(test_vector):
#                     word_prob = self.word_probabilities[label][i]
#                     if word_prob > 0:
#                         class_probabilities.append(word_count * math.log(word_prob))                
#                 results[label] = math.log(self.priors[label]) + sum(class_probabilities)
            
#             max_key = max(results, key=lambda k: results[k])
#             predictions.append(max_key)
        
#         return predictions


In [ ]:
# classifier = NaiveBayesTextClassifier()
# classifier.fit(train['text'], train['label'])

In [ ]:
# prediction = classifier.predict(test['text'])

In [ ]:
# prediction